In [1]:
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm
tqdm.pandas()
import warnings
warnings.filterwarnings('ignore')

In [2]:
%%time
train = pd.read_pickle('./data/train.pkl')
test = pd.read_pickle('./data/test.pkl')
data = pd.read_pickle('./data/data.pkl')

CPU times: user 22.7 s, sys: 25 s, total: 47.7 s
Wall time: 4min 58s


In [ ]:
!mkdir ./idlist

for col in tdqm(['creative_id', 'ad_id', 'product_id', 'product_category','advertiser_id','industry','click_times']):
    tiny_data = data[['user_id', col]]
    if col not in ['time','click_times']:
        val_cnt = tiny_data[col].value_counts()
        id_map = {} 
        differ = set(train[col].unique()).symmetric_difference(set(test[col].unique()))
        common = set(train[col].unique()) and (set(test[col].unique()))
        for v in differ: #训练集测试集不一样的也统一起来当一个id
            id_map[v] = 0
        for i, v in enumerate(common): #相同的按index累加当id
            id_map[v] = i + 1
        for v in val_cnt[val_cnt == 1].index: #出现一次的统一起来当成一个id
            id_map[v] = 0
        tiny_data[col] = tiny_data[col].map(id_map)
    
    list_d = tiny_data.groupby('user_id', sort=False)[col].progress_apply(list).reset_index()
    list_d.columns = ['user_id', col]
    list_d = list_d.sort_values(by='user_id').reset_index(drop=True)
    list_d[col].to_pickle('./idlist/{}_list.pkl'.format(col))